#Prepare environment

In [29]:
%matplotlib inline

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #pour faire des graph dans la bonne couleur etc.. par default
import glob
import re

#Upload datasets

> Define products to be taken into account for credited conversions

>/!\ Which products do you want to take into account????

In [31]:
#http://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
#gamme_prod = ['DN', 'DI', 'PD', 'TI', 'TN']
gamme_prod = ['DN', 'DI', 'PD', 'TI', 'TN','Mixte']

> Name of the files to be uploaded

> These should be hosted in the root folder

In [32]:
#file_quanti 
adServerFile = 'Report_Quantcast_mobilite_Standard_v1_M88Z5cUHryFJpRP_AA1S.xlsx'
#file_quali 
detailedFile = 'Report_Quantcast_mobilite_conversion_view_PNJjl0461vpIzpMgHrA7.csv'

In [33]:
#first file to be uploaded details of products part of each conversion
detailedConversions = pd.read_csv(detailedFile, dtype=object)

#We need to update the date format and strip hours
#http://stackoverflow.com/questions/26387986/strip-time-from-an-object-date-in-pandas
detailedConversions['conversion_date'] = pd.to_datetime(detailedConversions['Conversion date-hour'])
detailedConversions['conversion_date'] = detailedConversions['conversion_date'].apply(lambda x:x.date().strftime('%Y-%m-%d'))

In [34]:
#detailedConversions details products within each orders
#check what's in detailedConversions
detailedConversions.count()

Conversion date-hour      3724
Conversion ID             3724
Conversion label          3724
Project ID                3724
Project                   3724
Channel ID                3724
Channel                   3724
Campaign ID               3724
Campaign                  3724
Campaign labels           3724
Ad network ID             3724
Ad network                3724
Site-Offer ID             3724
Site-Offer                3724
Insertion ID              3724
Insertion                 3724
Creative ID               3724
Creative                  3724
Creative Sizes            3724
Creative Dimension        3724
Keyword                   3724
Text ad                   3724
Network                   3724
Website                   3724
Last action name          3724
Last action date-hour     3724
Match Type                3724
Device                    3724
Order number              3724
Client ID                 3724
New Client                3724
Revenues                  3724
Items nu

In [35]:
#then upload the second file. this is the regular 3PAS files with every conversions to which we contributed. 
xls_file = pd.ExcelFile(adServerFile)
adServerTable = xls_file.parse('DataView')

In [36]:
#adServerTable credits each order to a vendor
# /!\ We want to had details of orders containing products for which we are credited /!\
#check what's in adServerTable
adServerTable.count()

Date                                          332
Site/Offer ID                                 332
Site/Offer                                    332
Insertion ID                                  332
Insertion                                     332
Creative ID                                   332
Creative                                      332
Creative Sizes                                332
Imp.                                          332
Clicks                                        332
PC conv. BNUM_Confirmation achat_Pro          332
PC conv. BNUM_Confirmation achat_Part         332
PI conv. BNUM_Confirmation achat_Pro          332
PI conv. BNUM_Confirmation achat_Part         332
Revenues (PC) BNUM_Confirmation achat_Pro     332
Revenues (PC) BNUM_Confirmation achat_Part    332
Revenues (PI) BNUM_Confirmation achat_Pro     332
Revenues (PI) BNUM_Confirmation achat_Part    332
dtype: int64

In [37]:
#create functio that will be used to create matching key by concatenating several columns
#http://stackoverflow.com/questions/23444858/concatenate-column-values-in-pandas-dataframe-with-nan-values
def concat(*args):
    strs = [str(arg) for arg in args if not pd.isnull(arg)]
    return '_'.join(strs) if strs else np.nan

np_concat = np.vectorize(concat)

In [38]:
#We need to define the key to join both file. 
#first 3PAS report Key
adServerTable['key'] = np_concat(adServerTable['Date'], adServerTable['Site/Offer ID'], adServerTable['Insertion ID'], adServerTable['Creative ID'])
#QA the created key
adServerTable[['Date','Site/Offer ID','Insertion ID','Creative ID', 'key']].head(3)

,Date,Site/Offer ID,Insertion ID,Creative ID,key
0,2016-03-08,1123,7490,1731,2016-03-08_1123_7490_1731
1,2016-03-08,1123,7483,1733,2016-03-08_1123_7483_1733
2,2016-03-09,1123,7492,1735,2016-03-09_1123_7492_1735


In [39]:
detailedConversions['key'] = np_concat(detailedConversions['conversion_date'], detailedConversions['Site-Offer ID'], detailedConversions['Insertion ID'], detailedConversions['Creative ID'])
#QA the created key
detailedConversions[['conversion_date','Site-Offer ID','Insertion ID','Creative ID', 'key']].head(3)

,conversion_date,Site-Offer ID,Insertion ID,Creative ID,key
0,2016-03-10,1123,7485,1731,2016-03-10_1123_7485_1731
1,2016-03-10,1123,7485,1731,2016-03-10_1123_7485_1731
2,2016-03-10,1123,7485,1731,2016-03-10_1123_7485_1731


> Let's define several functions that'll be used for cleaning datasets

In [40]:
# function aimed at cleaning revenue figures

def cleanrev(col):
    if col == '-':
        return 0.0
    else:
        return float(col)

In [41]:
# function aimed at cleaning products part of the orders

def cleanGamme(col,gamme_prod):
    items = list(set(col.split("|")) )
    if len(items) == 1:
        return items[0]
    elif re.search('|'.join(gamme_prod),col):
        return "Mixte"
    else:
        return "Mixte_non_comptabilise"
        

In [42]:
# function aimed at associating products and unit price
# /!\ We need to verify every product are added and quantity matches

def cleanProduitMix(df):
    items = df[0].split("|")
    prices = df[1].split("|")
    quantity = df[2].split("|")
    if len(items) == 1:
        return items[0] + '-%s' %float(prices[0]) #*float(quantity[0]))
    else:
#        mix = '%s-' %len(items)
        l = len(items)
        mix = ''
        for i in range(0,len(items)-2):
            mix = mix + items[i] + '-%s|' %float(prices[i]) #*float(quantity[i]))
        return mix + items[l-1] + '-%s' %float(prices[l-1]) #*float(quantity[l-1]))

In [43]:
#function aimed at concatenating several columns separated by '-'
def concatenate(df):
    if len(df)>0:
        c = ''
        l = len(df)-1
        for i in range (0, l-1):
            c = c + df[i] + '-'
        return c + df[l]
    else:
        return ''

> The goal here is to agregate orders constituted by credited products to the 3PAS file

In [44]:

details3PAS = detailedConversions[['key','nom_site','gamme_produit', 'prix_produit','nombre_produit' ,'prix_ht_zanox']]
details3PAS = pd.DataFrame(details3PAS).dropna()
    
details3PAS['revenue_transaction'] = details3PAS.prix_ht_zanox.apply(lambda x: cleanrev(x))
details3PAS['panier'] = details3PAS.gamme_produit.apply(lambda x: cleanGamme(x,gamme_prod))

details3PAS.head(4)

,key,nom_site,gamme_produit,prix_produit,nombre_produit,prix_ht_zanox,revenue_transaction,panier
0,2016-03-10_1123_7485_1731,Part,PD,37.50,1,37.50,37.50,PD
1,2016-03-10_1123_7485_1731,Pro,MTEL,17.00,1,17.00,17.00,MTEL
2,2016-03-10_1123_7485_1731,Part,MTEL,16.32,24,16.32,16.32,MTEL
3,2016-03-10_1123_7492_1735,Part,Produit_physique_part|Produit_physique_part|Pr...,3.55|1.96|1.25|4.56|4.10|0.95|0.68|0.68|0.68|1...,1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|...,48.21,48.21,Mixte_non_comptabilise


In [49]:
#details3PAS[details3PAS.panier == 'Mixte'].head(4)

,key,nom_site,gamme_produit,prix_produit,nombre_produit,prix_ht_zanox,revenue_transaction,panier
188,2016-03-14_1123_7491_1740,Part,Produit_physique_part|Produit_physique_part|DN,8.40|8.40|25.50,1|1|1,42.30,42.30,Mixte
209,2016-03-15_1123_7485_1731,Part,VREL|DN,6.91|46.00,1|1,52.91,52.91,Mixte


# /!\  Section to be used once files are correctly populated

In [82]:
#reorder products with unit prices

#http://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe
#http://stackoverflow.com/questions/19914937/applying-function-with-multiple-arguments-to-create-a-new-pandas-column

details3PAS['order'] = details3PAS[['gamme_produit','prix_produit','nombre_produit']].apply(cleanProduitMix, axis = 1)
details3PAS = details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
details3PAS.head(5)

,key,revenue_transaction,panier,order
0,2016-03-10_1123_7485_1731,37.50,PD,PD-37.5
1,2016-03-10_1123_7485_1731,17.00,MTEL,MTEL-17.0
2,2016-03-10_1123_7485_1731,16.32,MTEL,MTEL-16.32
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-3.55|Produit_physique_pa...
4,2016-03-10_1123_7487_1735,58.50,PD,PD-58.5


In [83]:
# It seems files are not accurately populated. 
# As basket details don't match actual basket (sum does not equals revenue transaction) we cannot use Mixed baskets
details3PAS[details3PAS.panier == 'Mixte'].head(10)

,key,revenue_transaction,panier,order
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-3.55|Produit_physique_pa...
82,2016-03-13_1123_7487_1735,23.28,Mixte,LRE-6.48
188,2016-03-14_1123_7491_1740,42.30,Mixte,Produit_physique_part-8.4|DN-25.5
198,2016-03-15_1123_7485_1731,40.59,Mixte,Produit_physique_part-5.6|AELC-12.15|AELC-6.1|...
209,2016-03-15_1123_7485_1731,52.91,Mixte,DN-46.0
352,2016-03-16_1123_7487_1735,40.64,Mixte,AELC-7.5|Produit_physique_part-5.44|Produit_ph...
464,2016-03-18_1123_7485_1731,71.81,Mixte,Produit_physique_part-8.4|Produit_physique_par...
480,2016-03-18_1123_7873_1732,43.50,Mixte,Produit_physique_part-9.6|Produit_physique_par...
513,2016-03-19_1123_7873_1732,81.88,Mixte,MTEL-35.64|MTEL-4.15|MTEL-8.04|MTEL-6.45|AELC-...
1048,2016-03-25_1123_7485_1731,8.48,Mixte,LRE-6.73


> We need to create a row per product purchased

In [84]:
#http://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows
#http://stackoverflow.com/questions/33622470/fast-way-to-split-column-into-multiple-rows-in-pandas

s = details3PAS['order'].str.split("|", expand=True).stack()
i = s.index.get_level_values(0)
df = details3PAS.loc[i].copy()
df["order"] = s.values
df.head(20)

,key,revenue_transaction,panier,order
0,2016-03-10_1123_7485_1731,37.50,PD,PD-37.5
1,2016-03-10_1123_7485_1731,17.00,MTEL,MTEL-17.0
2,2016-03-10_1123_7485_1731,16.32,MTEL,MTEL-16.32
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-3.55
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-1.96
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-1.25
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-4.56
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-4.1
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-0.95
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part-0.68


In [221]:
df = df.drop(['gamme_produit','prix_produit', 'prix_ht_zanox'], 1)

s = df.order.apply(lambda x: pd.Series(x.split('-')))

df = df.drop(['order'], 1)

# prepare columns name for when we split order column
c = df.columns.values
c = np.append('key','item','unit_price'], axis = 0)

df = df.join(s)
df.columns =  c

df.head(15)

,key,revenue_transaction,panier,item,unit_price
0,2016-03-10_1123_7485_1731,37.50,PD,PD,37.50
1,2016-03-10_1123_7485_1731,17.00,MTEL,MTEL,17.00
2,2016-03-10_1123_7485_1731,16.32,MTEL,MTEL,16.32
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,3.55
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,1.96
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,1.25
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,4.56
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,4.10
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,0.95
3,2016-03-10_1123_7492_1735,48.21,Mixte,Produit_physique_part,0.68


In [86]:
df.tail(3)

,key,revenue_transaction,panier,order
3721,2016-04-10_1123_7484_1733,25.50,DN,DN-25.5
3722,2016-04-10_1123_7873_1732,6.10,AELC,AELC-6.1
3723,2016-04-10_1123_7485_1731,7.67,LRE,LRE-7.67


> Performances summary

In [186]:
df.count()

key                    4223
gamme_produit          4223
prix_produit           4223
prix_ht_zanox          4223
revenue_transaction    4223
panier                 4223
order                  4223
item                   4223
unit_price             4223
dtype: int64

# /!\ filter dataset

In [45]:
details3PAS.count()

key                    3647
nom_site               3647
gamme_produit          3647
prix_produit           3647
nombre_produit         3647
prix_ht_zanox          3647
revenue_transaction    3647
panier                 3647
dtype: int64

In [46]:
dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
dataSet3PAS.head(3)

,key,nom_site,revenue_transaction,panier
0,2016-03-10_1123_7485_1731,Part,37.50,PD
1,2016-03-10_1123_7485_1731,Pro,17.00,MTEL
2,2016-03-10_1123_7485_1731,Part,16.32,MTEL


In [53]:
dataSet3PAS = dataSet3PAS[dataSet3PAS.panier.isin(gamme_prod)]
dataSet3PAS.head(5)

,key,nom_site,revenue_transaction,panier
0,2016-03-10_1123_7485_1731,Part,37.5,PD
4,2016-03-10_1123_7487_1735,Part,58.5,PD
7,2016-03-10_1123_7490_1731,Part,25.5,DN
9,2016-03-10_1123_7492_1735,Part,25.5,DN
13,2016-03-10_1123_7485_1731,Part,25.5,DN


In [54]:
transactions = dataSet3PAS.groupby(['key']).count()
revenue = dataSet3PAS.groupby(['key']).sum()
ds = pd.DataFrame(transactions,revenue)

In [55]:
transactions = pd.DataFrame(dataSet3PAS.groupby(["key"]).sum()["revenue_transaction"])
revenue = pd.DataFrame(dataSet3PAS.groupby(["key"]).count()["panier"])
ds = transactions.join(revenue).reset_index()
#c = np.append('key', 'revenue', 'conversions',axis =0)
ds.columns = ['key', 'revenue', 'conversions']
ds.head(5)

,key,revenue,conversions
0,2016-03-10_1123_7485_1731,63.0,2
1,2016-03-10_1123_7487_1735,58.5,1
2,2016-03-10_1123_7490_1731,25.5,1
3,2016-03-10_1123_7492_1735,25.5,1
4,2016-03-11_1123_7484_1733,37.5,1


In [56]:
dataSet3PAS[dataSet3PAS.key == '2016-03-10_1123_7485_1731']

,key,nom_site,revenue_transaction,panier
0,2016-03-10_1123_7485_1731,Part,37.5,PD
13,2016-03-10_1123_7485_1731,Part,25.5,DN


In [57]:
ds[ds.key == '2016-03-10_1123_7485_1731']

,key,revenue,conversions
0,2016-03-10_1123_7485_1731,63,2


In [58]:
final_ds = pd.merge(adServerTable,ds, on='key')

In [59]:
final_ds[final_ds.key == '2016-03-10_1123_7485_1731'].head(5)

,Date,Site/Offer ID,Site/Offer,Insertion ID,Insertion,Creative ID,Creative,Creative Sizes,Imp.,Clicks,...,PC conv. BNUM_Confirmation achat_Part,PI conv. BNUM_Confirmation achat_Pro,PI conv. BNUM_Confirmation achat_Part,Revenues (PC) BNUM_Confirmation achat_Pro,Revenues (PC) BNUM_Confirmation achat_Part,Revenues (PI) BNUM_Confirmation achat_Pro,Revenues (PI) BNUM_Confirmation achat_Part,key,revenue,conversions
3,2016-03-10,1123,Quantcast,7485,Conquête pack déménagement Particuliers_300x250,1731,300x250_LaPoste_Demenagement,300x250,17539,2,...,1,1,5,-,37.5,17,67.05,2016-03-10_1123_7485_1731,63,2


In [60]:
final_ds = final_ds[['Date', 'Site/Offer ID','Site/Offer', 'Insertion ID', 'Insertion', 'Creative ID', 'Creative', 'Creative Sizes','Imp.', 'Clicks','conversions', 'revenue', 'key' ]]

In [61]:
final_ds.head(4)

,Date,Site/Offer ID,Site/Offer,Insertion ID,Insertion,Creative ID,Creative,Creative Sizes,Imp.,Clicks,conversions,revenue,key
0,2016-03-10,1123,Quantcast,7492,Retargeting pack déménagement Particuliers_728x90,1735,728x90_LaPoste_Demenagement,728x90,30698,7,1,25.5,2016-03-10_1123_7492_1735
1,2016-03-10,1123,Quantcast,7487,Conquête pack déménagement Particuliers_728x90,1735,728x90_LaPoste_Demenagement,728x90,22671,3,1,58.5,2016-03-10_1123_7487_1735
2,2016-03-10,1123,Quantcast,7490,Retargeting pack déménagement Particuliers_300...,1731,300x250_LaPoste_Demenagement,300x250,8422,3,1,25.5,2016-03-10_1123_7490_1731
3,2016-03-10,1123,Quantcast,7485,Conquête pack déménagement Particuliers_300x250,1731,300x250_LaPoste_Demenagement,300x250,17539,2,2,63.0,2016-03-10_1123_7485_1731


In [62]:
final_ds.to_csv('3pas_report_final.csv', index=False,encoding='utf-8')
print('Done')

Done
